<a href="https://colab.research.google.com/github/AfsanehHabibi/reddit-conversation-quality/blob/main/Sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install anytree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 577.3 kB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
base_path = "/content/drive/MyDrive/University/RedditData/"

In [5]:
import pickle

with open(f"{base_path}conversations_readability.pkl", 'rb') as file:
    conversations = pickle.load(file)

print("Len loaded data:", len(conversations))

Len loaded data: 32990


In [10]:
for id in conversations:
  print(conversations[id]['comments'][0])
  print(conversations[id]['pairs'][0])
  break

{'id': 'xyt38s', 'body': 'MLB Sat Info\n\nPlay\xa0OnAll underdogs with a money line of +100 or higher\xa0(SEATTLE)team with a terrible SLG (&lt;=.400) against a very good starting pitcher (WHIP&lt;=1.300) -AL, with a very good bullpen whose WHIP is 1.250 or better on the season217-224\xa0over the last 5 seasons.49.2%\xa0(89.1\xa0units)', 'author': None, 'reply_to': None, 'conversation_id': 'xyt38s', 'politness_score': 0.09523809523809523, 'readability_score': -0.5}
{'comment': 'MLB Sat Info\n\nPlay\xa0OnAll underdogs with a money line of +100 or higher\xa0(SEATTLE)team with a terrible SLG (&lt;=.400) against a very good starting pitcher (WHIP&lt;=1.300) -AL, with a very good bullpen whose WHIP is 1.250 or better on the season217-224\xa0over the last 5 seasons.49.2%\xa0(89.1\xa0units)', 'reply': 'Play\xa0OnAll underdogs with a money line of +100 or higher\xa0(TAMPA BAY)team with a terrible SLG (<=.400) against a very good starting pitcher (WHIP<=1.300) -AL, with a very good bullpen whos